In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
train = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)
print(train.shape)
train.head()

In [ ]:
test = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", quoting=3)
print(test.shape)
test.head()

In [ ]:
unlabeled_train = pd.read_csv("../input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)
print(unlabeled_train.shape)
#unlabeled_train.head()

In [ ]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
a = train['review'][0]
review_text = BeautifulSoup(a).get_text()
letters_only = re.sub("[^a-zA-Z]", " ", review_text)
words = letters_only.lower().split()
stop_words = set(stopwords.words("english"))
words1 = [w for w in words if w not in stop_words]

In [ ]:
#train['review'][0]

In [ ]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_words(review, remove_stop_words = False):
    
    review_text = BeautifulSoup(review).get_text()
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    words = letters_only.lower().split()
    if remove_stop_words:
        stop_words = set(stopwords.words("english"))
        words = [w for w in words if w not in stop_words]
    
    return words
    #return (" ".join(words))    

In [ ]:
import nltk.data
from nltk import word_tokenize,sent_tokenize  

def review_to_sentences( review, remove_stopwords=False ):
    raw_sentences = nltk.sent_tokenize(review)
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_words( raw_sentence, remove_stopwords ))
    return sentences

In [ ]:
sentences = []

for  review in train['review']:
    sentences += review_to_sentences(review)

In [ ]:
from gensim.models import Word2Vec, KeyedVectors
pretrained_path = "../input/googlewords/AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.bin"
model = KeyedVectors.load_word2vec_format(pretrained_path, binary = True, limit=500000)

In [ ]:
print("Dimension of the word 'tree': ", len(model["tree"]))

In [ ]:
len(train['review'])

In [ ]:
import time
start_time = time.time()

def convert_to_vectors(reviews, remove_stop_words = False):
    vectors = []
    for  i, review in enumerate(reviews):
        if (i + 1) % 2500 == 0:
            print("Review ", i+1)

        words = review_to_words(review, remove_stop_words)
        temp = []
        for word in words:
            try:
                arr = model[word]
                temp.append(arr)
            except:
                pass
        avg = np.mean(temp, axis = 0)
        vectors.append(avg)
    #vectors = np.array(vectors)
    print(len(vectors))
    print("Run time", time.time() - start_time)
    return vectors

In [ ]:
vectors = convert_to_vectors(train['review'],remove_stop_words = True)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(vectors, train['sentiment'], test_size = 0.1, random_state = 42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(verbose = 0, random_state = 0)
forest1 = RandomForestClassifier(verbose = 0, random_state = 0)

forest.fit(x_train, y_train)                # Use whie working
forest1.fit(vectors, train['sentiment'])    #Use for submission, train on full data

In [ ]:
y_pred= forest.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy= accuracy_score(y_test, y_pred)
print("Accuracy(in %):", accuracy)

In [ ]:
test_vectors = convert_to_vectors(test['review'],remove_stop_words = True)

In [ ]:
result = forest1.predict(test_vectors)

In [ ]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
output.to_csv("submission.csv", index = False, quoting=3)